# Pozyskanie danych - czytania podczas mszy

In [8]:
import pandas as pd

## Język polski 
Pobieranie czytań ze strony: http://mateusz.pl/czytania

In [65]:
import requests
from bs4 import BeautifulSoup

def get_addresses(n):
    addresses = []
    date_list = [base - datetime.timedelta(days=x) for x in range(0, n)]
    dates_cleaned = [str(x).replace('-','') for x in date_list]
    for date in dates_cleaned:
        ymd = date
        address = "http://mateusz.pl/czytania/" + ymd[:4]+ "/" + ymd + ".html"
        addresses.append(address)
    return addresses

def get_readings(address, day_name):
    page = requests.get(address)
    soup = BeautifulSoup(page.content, 'html.parser')
    #print(soup.prettify()) 
    #Exception handling for more readings
    try:
        #section = list(soup.children)[9]
        #section_content = section.find_all('p')   
        section = list(soup.children)
        #if len(section) != 30:
            #print(len(section), address, day_name)
        section_content1 = section[2].find_all('p')
        if ((day_name == 'Sunday') or (len(section) in [32,34])):
            section_content2 = section[6]
            section_content3 = section[8]
            section_content4 = section[12]
        else:
            section_content2 = None
            section_content3 = section[4]
            section_content4 = section[8]
            
        pierwsze_czytanie = str(section_content1[6])
        drugie_czytanie = str(section_content2)
        tekst_przed_ewangelia = str(section_content3)
        psalm_ref = str(section_content1[7])
        psalm = ""
        i = 8
        while i < len(section_content1):
            psalm += str(section_content1[i])
            i += 1
        ewangelia = str(section_content4)
        document = [pierwsze_czytanie,psalm_ref,psalm, drugie_czytanie, tekst_przed_ewangelia, ewangelia]
        return document
    except:
        pass
        #section = soup.find_all('section')[0]
        #section_content =section.find_all('p')

In [5]:
import datetime

#czytanie na dziś
now = datetime.datetime.now()
today = datetime.date.today()
year = str(now.year)
month = str(now.month)
if len(month) == 1:
    month = "0" + month

day = str(now.day)
if len(day) == 1:
    day = "0" + day

ymd = year+month+day
today_name = now.strftime("%A")
day_name = today_name
#today_name = 'Sunday'

address = "http://mateusz.pl/czytania/" + year + "/" + ymd + ".html"
#address = "http://mateusz.pl/czytania/2018/20181021.html" #niedziela
doc = get_readings(address, today)
#print(address)

In [9]:
df_raw = pd.DataFrame(data=None, index=None, #pd.date_range('1/1/2019', periods=10, freq='D'), 
                  columns = ["Dzień tygodnia","Pierwsze czytanie", "Psalm_ref", "Psalm", "Drugie czytanie", 
                                                    "Werset przed Ewangelią", "Ewangelia"])
df_raw.head()

,Dzień tygodnia,Pierwsze czytanie,Psalm_ref,Psalm,Drugie czytanie,Werset przed Ewangelią,Ewangelia


In [48]:
n=7
base = datetime.date.today()
date_list = [base - datetime.timedelta(days=x) for x in range(0, n)]
dates_cleaned = [str(x).replace('-','') for x in date_list]
dates_names = [x.strftime("%A") for x in date_list]
addresses = get_adresses(n)

for i in range(n):
        doc = get_readings(addresses[i], dates_names[i])
        #doc_cleaned = [clean_text(str(text)) for text in doc]
        input_row = [dates_names[i]]
        for each in doc: #doc_cleaned:
            input_row.append(each)
        df_raw.loc[date_list[i]] = input_row
df_raw.head()

,Dzień tygodnia,Pierwsze czytanie,Psalm_ref,Psalm,Drugie czytanie,Werset przed Ewangelią,Ewangelia
2019-03-29,Friday,"<p>\r\n (Oz 14, 2-10) <...","<p>\r\n (Ps 81 (80), 6c...","<p>Słyszę słowa nieznane:<br/>\r\n""Uwolniłem o...",None,<p>\r\n Aklamacja (Mt 4...,"<p>\r\n (Mk 12, 28b-34)..."
2019-03-28,Thursday,"<p>\r\n (Jr 7, 23-28) <...","<p>\r\n (Ps 95 (94), 1-...","<p>Przyjdźcie, radośnie śpiewajmy Panu,<br/>\r...",None,<p>\r\n Aklamacja (Jl 2...,"<p>\r\n (Łk 11, 14-23) ..."
2019-03-27,Wednesday,"<p>\r\n (Pwt 4, 1. 5-9)...","<p>\r\n (Ps 147B, 12-13...","<p>Chwal, Jeruzalem, Pana,<br/>\r\nwysławiaj t...",None,"<p>\r\n Aklamacja (J 6,...","<p>\r\n (Mt 5, 17-19) <..."
2019-03-26,Tuesday,"<p>\r\n (Dn 3, 25. 34-4...","<p>\r\n (Ps 25 (24), 4-...","<p>Daj mi poznać Twoje drogi, Panie,<br/>\r\nn...",None,<p>\r\n Aklamacja (Jl 2...,"<p>\r\n (Mt 18, 21-35) ..."
2019-03-25,Monday,"<p>\r\n (Iz 7, 10-14) <...","<p>\r\n (Ps 40, 7-8a. 8...",<p>Nie chciałeś ofiary krwawej ani płodów ziem...,"<p>\r\n (Hbr 10, 4-10) ...","<p>\r\n Aklamacja (J 1,...","<p>\r\n (Łk 1, 26-38) <..."


### Czyszczenie do formy tekstu zrozumiałego dla człowieka

In [17]:
import re

#first phase - for easy reading
def clean_html(raw):
    cleaner = re.compile('<.*?>')
    cleantext = re.sub(cleaner, '', raw)
    return cleantext

def clean_proverbs (raw):
    cleaner = re.compile('\([^)]*\)')
    cleantext = re.sub(cleaner, '',raw)
    return cleantext

def clean_indent(raw):
    cleaner = re.compile('^\s+')
    cleantext = re.sub(cleaner, '',raw)
    return cleantext

def clean_rest(text):
    cleaner = re.compile('\\r\\n\s+')
    cleantext = re.sub(cleaner, '',text)
    return cleantext

def clean_rest2(text):
    cleaner = re.compile('(\\r\\n)')
    cleantext = re.sub(cleaner, ' ',text)
    return cleantext

def clean_carriage(text):
    cleaner = re.compile('(.*)\n')
    cleantext = re.sub(cleaner, '',text)
    return cleantext

def clean_psalmus(text):
    cleaner = re.compile('Aklamacja|REFREN\:\s|Bracia\:\s')
    cleantext = re.sub(cleaner, '',text)
    return cleantext

def clean_text(raw):
    text = clean_html(raw)
    text = clean_proverbs(text)
    text = clean_indent(text)
    text = clean_rest(text)
    text = clean_rest2(text)
    text = clean_carriage(text)
    text = clean_psalmus(text)
    return text

In [22]:
df = df_raw.copy()
for each in df_raw.columns:
    df[each] = df[each].apply(lambda x: clean_text(x))
df.head(7)

,Dzień tygodnia,Pierwsze czytanie,Psalm_ref,Psalm,Drugie czytanie,Werset przed Ewangelią,Ewangelia
2019-03-29,Friday,"Tak mówi Pan: ""Wróć, Izraelu, do Pana, Boga tw...","Ja jestem Bogiem, słuchaj mego głosu","Słyszę słowa nieznane: ""Uwolniłem od brzemieni...",None,"Pan mówi: Nawracajcie się, bliskie jest król...",Jeden z uczonych w Piśmie podszedł do Jezusa i...
2019-03-28,Thursday,"Tak mówi Pan: ""Dałem im przykazanie: „Słuchajc...","Słysząc głos Pana, serc nie zatwardzajcie","Przyjdźcie, radośnie śpiewajmy Panu, wznośmy o...",None,"Nawróćcie się do Boga waszego, On bowiem jes...","Jezus wyrzucał złego ducha z człowieka, który ..."
2019-03-27,Wednesday,"Mojżesz powiedział do ludu: ""A teraz, Izraelu,...","Kościele święty, chwal swojego Pana","Chwal, Jeruzalem, Pana, wysławiaj twego Boga, ...",None,"Słowa Twoje, Panie, są duchem i życiem. Ty m...","Jezus powiedział do swoich uczniów: ""Nie sądźc..."
2019-03-26,Tuesday,"Powstawszy, Azariasz tak się modlił, a otwarłs...","Pamiętaj o nas, miłosierny Panie","Daj mi poznać Twoje drogi, Panie, naucz mnie c...",None,"Nawróćcie się do Boga waszego, On bowiem jes...","Piotr podszedł do Jezusa i zapytał: ""Panie, il..."
2019-03-25,Monday,Pan przemówił do Achaza tymi słowami: „Proś dl...,"Przychodzę, Boże, pełnić Twoją wolę","Nie chciałeś ofiary krwawej ani płodów ziemi, ...","Niemożliwe jest to, aby krew cielców i kozłów ...",Słowo stało się ciałem i zamieszkało między ...,Bóg posłał anioła Gabriela do miasta w Galilei...
2019-03-24,Sunday,Gdy Mojżesz pasł owce swego teścia imieniem Je...,"Pan jest łaskawy, pełen miłosierdzia","Błogosław, duszo moja, Pana, i wszystko, co je...","Nie chciałbym, bracia, byście nie wiedzieli, ż...","Pan mówi: Nawracajcie się, bliskie jest króles...",W tym czasie przyszli jacyś ludzie i donieśli ...
2019-03-23,Saturday,"Paś lud Twój, Panie, laską Twoją, trzodę dzied...","Pan jest łaskawy, pełen miłosierdzia","Błogosław, duszo moja, Pana, i wszystko, co je...",None,"Zabiorę się i pójdę do mego ojca, i powiem: ""...",W owym czasie przybliżali się do Jezusa wszysc...


### Przykład

In [44]:
df.set_index(pd.Index([str(x) for x in df.index]), inplace=True) # zamiania indeksu z formatu datetime() na str()
df.loc['2019-03-23', 'Ewangelia']

'W owym czasie przybliżali się do Jezusa wszyscy celnicy i grzesznicy, aby Go słuchać. Na to szemrali faryzeusze i uczeni w Piśmie, mówiąc: "Ten przyjmuje grzeszników i jada z nimi". Opowiedział im wtedy następującą przypowieść: "Pewien człowiek miał dwóch synów. Młodszy z nich rzekł do ojca: „Ojcze, daj mi część własności, która na mnie przypada”. Podzielił więc majątek między nich. Niedługo potem młodszy syn, zabrawszy wszystko, odjechał w dalekie strony i tam roztrwonił swoją własność, żyjąc rozrzutnie. A gdy wszystko wydał, nastał ciężki głód w owej krainie, i on sam zaczął cierpieć niedostatek. Poszedł i przystał na służbę do jednego z obywateli owej krainy, a ten posłał go na swoje pola, żeby pasł świnie. Pragnął on napełnić swój żołądek strąkami, którymi żywiły się świnie, lecz nikt mu ich nie dawał. Wtedy zastanowił się i rzekł: „Iluż to najemników mojego ojca ma pod dostatkiem chleba, a ja tu przymieram głodem. Zabiorę się i pójdę do mego ojca, i powiem mu: Ojcze, zgrzeszyłem 

### Zapis do pliku csv

In [49]:
df.to_csv(path_or_buf= 'czytania.csv', header=df.columns, sep=',')

## Język angielski
Pobieranie czytań ze strony: http://www.usccb.org/bible/readings/

In [174]:
address =  "http://www.usccb.org/bible/readings/"
print(address)

http://www.usccb.org/bible/readings/


In [180]:
def get_addresses(n):
    addresses = []
    date_list = [base - datetime.timedelta(days=x) for x in range(0, n)]
    dates_cleaned = [str(x).replace('-','') for x in date_list]
    for date in dates_cleaned:
        ymd = date
        address = "http://www.usccb.org/bible/readings/" + ymd[4:6] + ymd[-2:] + ymd[2:4] + ".cfm"
        addresses.append(address)
    return addresses

def get_readings(address):
    try:
        page = requests.get(address)
        soup = BeautifulSoup(page.content, 'html.parser')
        section = list(soup.children)[8]
        section = list(section.children) [15]
        first = section.find_all('div')[11]
        psalm = section.find_all('div')[15]
        vers = section.find_all('div')[19]
        gospel = section.find_all('div')[23]
        document = [first, psalm, vers, gospel]
        return document
    except:
        pass

In [191]:
df_raw = pd.DataFrame(data=None, index=None, #pd.date_range('1/1/2019', periods=10, freq='D'), 
                  columns = ["Dzień tygodnia","Pierwsze czytanie","Psalm","Werset przed Ewangelią", "Ewangelia"])
df_raw.head()

,Dzień tygodnia,Pierwsze czytanie,Psalm,Werset przed Ewangelią,Ewangelia


In [190]:
n=7
base = datetime.date.today()
date_list = [base - datetime.timedelta(days=x) for x in range(0, n)]
dates_cleaned = [str(x).replace('-','') for x in date_list]
dates_names = [x.strftime("%A") for x in date_list]
addresses = get_addresses(n)

for i in range(n):
        doc = get_readings(addresses[i])
        input_row = [dates_names[i]]
        for each in doc: 
            input_row.append(each)
        df_raw.loc[date_list[i]] = input_row
df_raw.head(7)

,Dzień tygodnia,Pierwsze czytanie,Psalm,Werset przed Ewangelią,Ewangelia
2019-03-29,Friday,"<div class=""poetry""> Thus says the LORD:<br/>R...","<div class=""poetry""> R. (see 11 and 9a) <stron...","<div class=""poetry""> Repent, says the Lord;<br...","<div class=""poetry""> One of the scribes came t..."
2019-03-28,Thursday,"<div class=""poetry""> Thus says the LORD: <br/>...","<div class=""poetry""> R. (8)<strong> If today y...","<div class=""poetry""> Even now, says the LORD,<...","<div class=""poetry""> Jesus was driving out a d..."
2019-03-27,Wednesday,"<div class=""poetry""> Moses spoke to the people...","<div class=""poetry""> R. (12a) <strong>Praise ...","<div class=""poetry""> Your words, Lord, are Spi...","<div class=""poetry""> Jesus said to his discipl..."
2019-03-26,Tuesday,"<div class=""poetry"">Azariah stood up in the fi...","<div class=""poetry""> R. (6a) <strong> Remember...","<div class=""poetry""> Even now, says the LORD,<...","<div class=""poetry"">Peter approached Jesus and..."
2019-03-25,Monday,"<div class=""poetry""> The LORD spoke to Ahaz, s...","<div class=""poetry""> R. (8a and 9a)<strong> He...","<div class=""poetry""> Brothers and sisters:<br/...","<div class=""bibleReadingsWrapper""> <h4>Gospel ..."
2019-03-24,Sunday,"<div class=""cs_control CS_Element_Textblock"" i...","<div id=""mc_embed_signup""> <form action=""//usc...","<div class=""clear"" id=""mce-responses""> <div cl...","<div class=""clear""><input class=""button"" id=""m..."
2019-03-23,Saturday,"<div class=""poetry""> Shepherd your people with...","<div class=""poetry""> R. (8a) <strong>The Lord...","<div class=""poetry""> I will get up and go to m...","<div class=""poetry""> Tax collectors and sinner..."


In [193]:
#to do: cleaning